# Lightweight Fine-Tuning Project

TODO: In this cell, describe your choices for each of the following

* PEFT technique: 
* Model: 
* Evaluation approach: 
* Fine-tuning dataset: 

## Loading and Evaluating a Foundation Model

TODO: In the cells below, load your chosen pre-trained Hugging Face model and evaluate its performance prior to fine-tuning. This step includes loading an appropriate tokenizer and dataset.

In [11]:
from datasets import load_dataset

dataset = load_dataset("sms_spam", split="train").train_test_split(
    test_size=0.2, shuffle=True, seed=23
)

splits = ["train", "test"]

dataset["test"]

Dataset({
    features: ['sms', 'label'],
    num_rows: 1115
})

In [12]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")


tokenized_dataset = {}
for split in splits:
    tokenized_dataset[split] = dataset[split].map(
        lambda x: tokenizer(x["sms"], truncation=True), batched=True
    )

tokenized_dataset["train"]

Dataset({
    features: ['sms', 'label', 'input_ids', 'attention_mask'],
    num_rows: 4459
})

In [13]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased",
    num_labels=2,
    id2label={0: "not spam", 1: "spam"},
    label2id={"not spam": 0, "spam": 1},
)


for param in model.parameters():
    param.requires_grad = False

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [14]:
print(model)

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): DistilBertSdpaAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)


In [15]:
import numpy as np
from transformers import DataCollatorWithPadding, Trainer, TrainingArguments


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return {"accuracy": (predictions == labels).mean()}


# The HuggingFace Trainer class handles the training and eval loop for PyTorch for us.
# Read more about it here https://huggingface.co/docs/transformers/main_classes/trainer
trainer = Trainer(
    model=model,
    args=TrainingArguments(
        output_dir="./data/spam_not_spam",
        # Set the learning rate
        learning_rate= 2e-3,
        # Set the per device train batch size and eval batch size
        per_device_eval_batch_size=4,
        per_device_train_batch_size=4,
        # Evaluate and save the model after each epoch
        eval_strategy="epoch",
        save_strategy="epoch",
        num_train_epochs=2,
        weight_decay=0.01,
        load_best_model_at_end=True,
    ),
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
    compute_metrics=compute_metrics,
)

trainer.evaluate()

C:\Users\rebjo\AppData\Local\Temp\ipykernel_24848\907536503.py:13: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


{'eval_loss': 0.6718908548355103,
 'eval_model_preparation_time': 0.0,
 'eval_accuracy': 0.7668161434977578,
 'eval_runtime': 2.2335,
 'eval_samples_per_second': 499.227,
 'eval_steps_per_second': 124.919}

## Performing Parameter-Efficient Fine-Tuning

TODO: In the cells below, create a PEFT model from your loaded model, run a training loop, and save the PEFT model weights.

In [27]:
from peft import LoraConfig, TaskType, get_peft_model

# peft_config = LoraConfig(
#     task_type=TaskType.SEQ_CLS, 
#     r=8,  
#     inference_mode=False,
#     lora_alpha=16,  
#     lora_dropout=0.1, 
#     target_modules=["pre_classifier", "classifier"]}
# )

# LoRA configuration
peft_config = LoraConfig(
    task_type=TaskType.SEQ_CLS, inference_mode=False, r=8, lora_alpha=32, lora_dropout=0.1,
    target_modules=["q_lin", "v_lin", "k_lin"]
)



In [28]:
model_lora = get_peft_model(model,peft_config)
model_lora.print_trainable_parameters()



trainable params: 813,314 || all params: 67,768,324 || trainable%: 1.2001


In [29]:
trainer_lora = Trainer(
    model=model_lora,
    args=TrainingArguments(
        output_dir="./data/spam_not_spam_lora",
        # Set the learning rate
        learning_rate= 2e-3,
        # Set the per device train batch size and eval batch size
        per_device_eval_batch_size=4,
        per_device_train_batch_size=4,
        # Evaluate and save the model after each epoch
        eval_strategy="epoch",
        save_strategy="epoch",
        num_train_epochs=2,
        weight_decay=0.01,
        load_best_model_at_end=True,
    ),
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
    compute_metrics=compute_metrics,
)

C:\Users\rebjo\AppData\Local\Temp\ipykernel_24848\277593029.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer_lora = Trainer(


In [30]:
model_lora.config.num_labels = 2
model_lora.config.id2label = {0: "not spam", 1: "spam"}
model_lora.config.label2id = {"not spam": 0, "spam": 1}

In [31]:
trainer_lora.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,0.485300,0.421397,0.870852
2,0.491500,0.590733,0.870852


TrainOutput(global_step=2230, training_loss=0.42643634128998215, metrics={'train_runtime': 91.4495, 'train_samples_per_second': 97.518, 'train_steps_per_second': 24.385, 'total_flos': 103147424907240.0, 'train_loss': 0.42643634128998215, 'epoch': 2.0})

In [32]:
model_lora.save_pretrained('distilbert-base-uncased_lora')

## Performing Inference with a PEFT Model

TODO: In the cells below, load the saved PEFT model weights and evaluate the performance of the trained PEFT model. Be sure to compare the results to the results from prior to fine-tuning.

In [40]:
from peft import AutoPeftModelForSequenceClassification
lora_model_ev = AutoPeftModelForSequenceClassification.from_pretrained('distilbert-base-uncased_lora')


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [41]:
trainer = Trainer(
    model=lora_model_ev,
    args=TrainingArguments(
        output_dir="./data/spam_not_spam_lora_evaluate",
        # Set the learning rate
        learning_rate= 2e-3,
        # Set the per device train batch size and eval batch size
        per_device_eval_batch_size=4,
        per_device_train_batch_size=4,
        # Evaluate and save the model after each epoch
        eval_strategy="epoch",
        save_strategy="epoch",
        num_train_epochs=2,
        weight_decay=0.01,
        load_best_model_at_end=True,
    ),
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
    compute_metrics=compute_metrics,
)

trainer.evaluate()

C:\Users\rebjo\AppData\Local\Temp\ipykernel_24848\3449228933.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


{'eval_loss': 0.42139726877212524,
 'eval_model_preparation_time': 0.0048,
 'eval_accuracy': 0.8708520179372198,
 'eval_runtime': 3.4757,
 'eval_samples_per_second': 320.799,
 'eval_steps_per_second': 80.272}